<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0129DETR_fine_turing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- original: [Transformerを使った初めての物体検出「DETR」第2回 アーキテクチャの詳細解説とFine-Tuning](https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/part2.html)
- author: オージス総研 技術部 データエンジニアリングセンター 堀 裕太
- date: 2021年11月25日



# 3. DETR のアーキテクチャの詳細説明

第1回目の記事でも図示しましたが、[DETR の論文](https://arxiv.org/pdf/2005.12872.pdf) に記載されているアーキテクチャの図を、改めて下図に示します。
また、DETR の Transformer 部分のアーキテクチャ図も示します。
前回よりも細かい解説と各次元の要素数 (以下、shape)を明記しながら解説していきます。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-009.png" width="59%"><br/>

出典 : [End-to-End Object Detection with Transformers](https://arxiv.org/pdf/2005.12872.pdf)
</center>

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-010.png" width="39%"><br/>

出典 : [End-to-End Object Detection with Transformers](https://arxiv.org/pdf/2005.12872.pdf)
</center>


## 3-1.①backbone

前章で説明したTransformerと比較すると、「2-1.Embedding」 に相当します。
ここでは、主に2つの処理を行います。

1. インプット画像に対して CNN で畳み込みを行い、$d$ 次元の特徴マップに変換
2. ``Transformer`` のインプットにするために ``reshape``

CNN では、ImageNet で事前学習した [ResNet-50](https://arxiv.org/pdf/1512.03385.pdf) を TorchVision からインポートし、最終層を削除して $d$ 次元に変換する 2 次元畳み込み層を追加したネットワークを使います。
また、[第1回目の記事]() でも掲載した処理イメージの図を見た方が分かりやすいかと思いますので、再掲します。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-011.png" width="49%><br/>
</center>

バッチサイズを ``B``、入力画像の幅･高さ･チャネル数をそれぞれ ``W, H, C``, CNN 適用後の特徴マップの幅･高さ･チャネル数をそれぞれ ``W'、H'、d`` で表しています (論文では W’=W/32, H’=H/32, d=256)。
インプット画像の ``shape`` は ``(B, H, W, C)`` だったものが、最終的に ``(B, H'×W', d)`` になります。


## 3-2.②positional encoding

* **3-1.①backbone**  で生成した特徴マップに対して、位置情報を付与します。
上図の DETR の ``Transformer`` 部分のアーキテクチャを見ていただければ分かる通り、DETR には Positional encoding として ``Spatial positional encoding`` と ``Object queries`` の 2 種類が使われています。
また、インプット時に 1 回だけ ``Positional Encoding`` を足し合わせるのではなく、``Attention`` 実行前に毎回 ``Positional Encoding`` を足し合わせています。

* **(1) Spatial positional encoding**
空間位置エンコーディングと呼ばれます。
``Attention is All You Need`` の論文と同じく、正弦波の固定値を使います。
詳細は ``2-2.Positional Encoding``  をご確認ください。
``Encoder`` の ``Attention`` 前、``Decoder`` の ``SourceTarget-Attention`` の前に ``Encoder`` のアウトプットに対して足し合わせます。

* **(2) Object queries**
オブジェクトクエリと呼ばれます。
``Object queries`` の本来の役割は ``Decoder`` のインプットですが、``Decoder`` 側の ``Positional Encoding`` としても使われています (Object queries の詳細は 「3-4.④decoder」 で説明します)。
また、``Object queries`` は正弦波の固定値ではなく、初期値がランダムな学習パラメータになります。

なぜ ``Positional Encoding`` が 2 種類あって、``Spatial positional encoding`` には正弦波の固定値、``Object queries`` には初期値がランダムな学習パラメータを使っているのかというと、様々な組み合わせを実験した結果、この組み合わせが一番性能が高かったためです。
実験結果が DETR の論文に記載されているので、下図に示します。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-012.png" width="49%"><br/>

出典 : [End-to-End Object Detection with Transformers](https://arxiv.org/pdf/2005.12872.pdf)
</center>

上記表では、左側が ``Positional Encoding`` の組み合わせ、右側が性能を表しています。
``spatial pos. enc.`` の ``encoder`` が ``Encoder`` の ``Attention`` 前、``decoder`` が ``Decoder`` の``SourceTarget-Attention`` の前の ``Encoder`` のアウトプットに足し合わせる箇所を指します。
また、``output pos. enc.`` の ``decoder`` は ``Decoder`` 側の ``Positional Encoding`` のことです。
それぞれ色々なパターンを試しています。
``none``  は ``Positional Encoding`` 自体を使いません。
``sine at input``  は ``Attention`` 前ではなくインプット時に 1 回だけ正弦波の固定値を足し合わせます。
`` sine at attn.`` は ``Attention`` ごとに正弦波の固定値を付与。
``learned at input``  と ``learned at attn.`` は学習パラメータをそれぞれ使った場合を意味します。
最終的に、表の一番下の行の性能が一番良かったため、このような組み合わせで ``Positional Encoding`` を使うことになったわけです。
また、``Positional Encoding`` の ``shape`` は特徴マップと同じく ``(B, H'×W', d)`` になります。

## 5-1.Open Image Datasetのフォーマット変換

Fine-Tuning元の学習済みモデルには、第1回目の記事 でも紹介した、Model Zooに公開されている「DETR-DC5」を使います。
このモデルは COCO dataset を使って学習しているため、Fine-Tuning をするために COCO dataset 以外のデータセットを用意します。
COCO dataset と重複していないクラスラベルを持つデータセットを使いたいので、今回は Open Image Dataset 1 を使うことにします。
しかし Open Image Dataset はそのままでは DETR では使えません。
元々の DETR は COCO dataset しか対応していないため、COCO dataset format に変換する必要がありますので、フォーマット変換方法も合わせて紹介します。

まずは Open Image Dataset をダウンロードする方法です。
2021 年時点の Open Image Dataset の最新バージョンは v6 ですが、今回は v4 をダウンロードできるツールである OIDv4 ToolKit 7 を利用します。
本来は 500GB 超えのデータセットをダウンロードする必要があるので Colab 上だと展開できないですが、ツールを使うことで必要なクラスラベルを必要な枚数だけダウンロードすることができるので、Colab 上でも問題なくダウンロードすることができます。

では、早速 OIDv4 ToolKit の Git リポジトリをクローンして必要なパッケージをインストールします。
requirements.txt だけではバージョンの互換性エラーが出るため、urllib3 と folium はバージョンを指定して個別にインストールしています。


## 3-3.③encoder

DETR の ``Transformer`` アーキテクチャ図の左側部分に相当します。
``2-9.Transformer``  で解説した内容とほぼ同じ仕組みですが、DETR では ``Self-Attention`` ごとに ``Positional Encoding`` を足し合わせる点が異なります。

補足になりますが、DETR の論文では ``Encoder`` の ``Self-Attention`` を可視化していたので、下図に示します。
それぞれ赤点から見て、各地点の ``Attention Score`` の高い箇所を黄色く表示したものになります。
下図を見たら分かるように、``Encoder`` の時点である程度オブジェクトを分離できていることが分かります。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-013.png" width="66%"><br/>

出典 : [End-to-End Object Detection with Transformers](https://arxiv.org/pdf/2005.12872.pdf)
</center>

## 3-4.④decoder

DETR の Transformer アーキテクチャ図の右側部分に相当します。
``Encoder`` と同様、基本は ``2-9.Transformer``  で解説した内容とほぼ同じです。
違いはインプットに ``Object queries`` を使うことと、自己回帰構造ではなくなったので並列的に処理できるようになったことです(並列デコーダについては後述)。

``Object queries`` とは何かというと、主な役割は ``Decoder`` へのインプットで、その実態は画像内から物体検出とラベル分類をするために学習されるパラメータのことです。
また、前述の通り ``Decoder`` の ``Positional Encoding`` として使う、という 2 種類の役割を持っています。
初期値はランダムなベクトル値であり、$d$ 次元のベクトルが任意の $N$ 個 (論文では $N=100$) で、それら全てが学習パラメータになります。
下図は物体検出結果から、対応する ``Object queries`` のうち 20 個を可視化したものです (``Object queries`` 自体のベクトルを可視化したものではありません。
``Decoder`` のアウトプットの ``Object queries`` を FFN に通して、バウンディングボックスを検出した結果を可視化しています)。
バウンディングボックスの中心位置に点を図示しており、赤色の点は大きな横長のバウンディングボックス、青色の点は大きな縦長のバウンディングボックス、緑色は小さなバウンディングボックスを表します。
``Object queries`` の 1 つ 1 つが別々のエリアやボックスサイズに特化して学習しているので、$N$ 個の物体を検出できるということになります。
そのため、100 個以上の物体を検出したい場合は $N>100$ にする必要があります。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-014.png" width="66%"><br/>

出典 : [End-to-End Object Detection with Transformers](https://arxiv.org/pdf/2005.12872.pdf)
</center>

``Encoder`` の補足と同様に、``Decoder`` の ``Attention`` を可視化したものを下図に示します。
可視化した箇所は象の鼻など色がついて光っている箇所で、バウンディングボックスではないことにご注意ください。
また、``Object queries`` ごとに別の色で可視化しています。
``Encoder`` で大まかなオブジェクト分離は完了しているため、``Decoder`` ではオブジェクト境界 (足や耳など) を集中して注目していることが分かります。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-015.png" width="66%"><br/>

出典 : [End-to-End Object Detection with Transformers](https://arxiv.org/pdf/2005.12872.pdf)
</center>

また、``Decoder`` のアウトプットの ``shape`` はインプットの ``shape`` と同じになるため、``(B, N, d)`` になります。

## 3-5.⑤prediction heads

``Transformer`` の ``Decoder`` で出力された予測済みの ``Object queries`` を、FFN に通してバウンディングボックスの座標とクラスラベルに変換します。
FFN は、バウンディングボックス側は ``Linear(ReLU) - Linear`` の 2 層ネットワーク、クラスラベル側は ``Linear`` 層によって計算されます。
ここで注意なのが、結果が必ず $N$ 個出力される点です。
$N$ は通常、画像内の実際のオブジェクト数よりも多く設定するので、大半はオブジェクトが検出されないことになります。
そのため、オブジェクトに紐付かないことを示すクラスラベル ∅ (no object) を設定します。
また、学習時に予測結果と正解を紐づける必要がありますが、紐付け方法は ``4-2.2部マッチングロス (Bipartite Matching Loss)`` で後述します。


# 4.DETRで使われた新技術

2 章と 3 章では、元々の ``Transformer`` と DETR について詳しく解説しました。
本章では、DETR から新たに使われている技術について紹介します。

## 4-1.並列デコーダ (Parallel Decoding)

元々の ``Transformer`` の ``Decoder`` は ``2-9.Transformer``  で解説した通り、自己回帰で単語を生成したり予測を行います。
そのため、出力データ長に比例して推論コストが非常に高くなってしまいます。
しかし DETR では、``Object queries`` を使うことによって、回帰構造ではなく並列処理ができる ``Decoder`` にすることができました。

## 4-2.2部マッチングロス (Bipartite Matching Loss)

DETR に画像を入力すると、``Object queries`` の数 $N(=100)$ 個のバウンディングボックスとクラスラベルが出力されます。
DETR で学習する際には、この出力結果と正解ラベルとを比較してロスを計算する必要があります。 
例えば下の図のように $N=4$ の時を考えた場合、それぞれ対応するバウンディングボックスに対してロス計算をしていきます。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-016.png" width="66%"><br/>
</center>

上記画像のように学習がある程度進んでいれば対応付けは容易ですが、学習初期などは出力結果は滅茶苦茶になっていると考えられます。
また、$N=100$ など個数が多くなってくると、出力結果と正解ラベルとの紐付けが非常に困難です。
この問題を ``2部マッチング問題`` と言います。
DETR では、``2部マッチング問題`` を効率的に解くために ``ハンガリアンアルゴリズム``を使います。
ハンガリアンアルゴリズムは最も効率の良いマッチングパターンを見つける手法であり、計算コストも $O(n!)$ から $O(n^{3})$ になります。

以下に、実際にハンガリアンアルゴリズムを使ったロス計算の数式を紹介します。
最初はハンガリアンアルゴリズムを使って、正解と推論結果の組み合わせでロス総和が最小になるようなマッチングパターン $\hat{\sigma}$ を求める数式です

$$
\hat{\sigma} =\arg\min_{\sigma\in S_{N}}\sum_{i}^{N} L_{\text{match}}\left(y_{i}, \hat{y}_{\sigma(i)}\right) \tag{1}
$$

$y_{i}$ は正解、$\hat{y}_{i}$ は推論結果を表します。
また $y_{i}=\{c_{i}, b_{i}\}$ となり、それぞれ $c_{i}$ は正解のクラスラベル、$b_{i}$ は正解のバウンディングボックス位置を表します ($\hat{c}_{i}$ と $\hat{b}_{i}$ も同じく、推論結果のクラスラベルとバウンディングボックス位置です)。
$N$ は ``Object queries`` の数です。
また、$S_{N}$ は $N$ 次の対称群を表します。
対称群とは集合の用語で、DETR では正解と推論結果の組み合わせが存在する群を示しています。
つまり $\sigma(i)$ は、正解の $i$ 番目に対応する推論結果のインデックス番号を返します。
数式の(1) 部分は、$S_{N}$ 上で (2) 部分が最小となるマッチングパターン $\sigma$ の集合という意味になります。
数式の(2) 部分は、ロス $L_{\text{match}}$ の総和を意味します。

損失 $L_{\text{match}}$ の数式は以下の通りです。
左項はクラスラベルの損失、右項はバウンディングボックス位置の損失に相当します。
ここで正解 $y_{i}$ と推論結果 $\hat{y}_{\sigma(i)}$ をマッチングした時に生じるロスを計算しています。

$$
L_{\text{match}}(y_{i},\hat{y}_{\sigma(i)})=\mathbb{1}_{c_{i}\ne \Phi}(c_{i})+\mathbb{1}_{c_{i}\ne\Phi}L_{\text{box}}(b_{i},\hat{b}_{\sigma(i)})\tag{2}
$$

太文字の $\mathbb{1}$ は集合を表していて、数式の (3) 部分は、該当クラスなし (no object) の場合は 0 の集合、それ以外は 1 の集合にするという意味です。
また、数式の (4) 部分では正解クラスラベルである確率のことを表します。
右項も同様に、該当クラスなし (no object) の場合は 0 の集合にして、それ以外の場合は $L_{\text{box}}$ を計算します。

$L_{\text{box}}$ の数式は以下になります。
バウンディングボックス位置に対する損失を表していて、左項では物体予測位置に対する損失である ``Generalized IoU`` (以下 ``GIoU``)損失、右項は ``L1 損失`` を計算しています。
バウンディングボックス位置に対する損失の計算には、通常は $L^{P}$ 損失が使われますが、大きな物体は損失が大きくなり、小さな物体は損失が小さくなってしまいます。
そのため、スケール依存性を小さくするために、``GIoU 損失`` と ``L1 損失`` を足し合わせています。

$$
L_{\text{box}}\left(b_{i},\hat{b}_{\sigma(i)}\right)=
\lambda_{\text{iou}} L_{\text{iou}}\left(b_{i},\hat{b}_{\sigma(i)}\right)
+\lambda_{L1}\left\|b_{i}-\hat{b}_{\sigma(i)}\right\|_{1}\tag{3}
$$

$\lambda_{\text{iou}}$ と $\lambda_{L1}$ はハイパーパラメータです。
数式の (6) 部分では L1ノルム (各成分の絶対値の和) を計算しています。
(5)部分は GIoU 損失関数で、数式は以下の通りです。
なお、 GIoU の数式をひとまとめにして記述すると分かり難いため、GIoU(A,B) として別式で記載しています。


$$
L_{iou}\left(b_{i},\hat{b}_{\sigma(i)}\right)=1\text{GIoU}\left(b_{i},\hat{b}_{\sigma(i)}\right)
$$

$$
\text{GIoU}(A,B)=\frac{\left|A\cap B\right|}{\left|A\cup B\right|}-
\frac{\left|C\neg (A\cup B)\right|}{\left|C\right|}
$$

$cap$ は積集合、$\cup$ は和集合、バックスラッシュは差集合を表しています。
また、$A$ は $b_{i}$， $B$ は $b_{\sigma(i)}$，$C$ は $A$ と $B$ の領域を囲う最小の矩形領域を示しています。

ここまでの数式で、損失の総和が最小になるようなマッチングパターン $\hat{\sgima$$ を求めることができます。
このマッチングパターン $\hat{\sigma}$ を使い、ハンガリアン損失を以下の数式で求めます。

$$
L_{\text{hungarian}}(y,\hat{y})=\sum_{i=1}^{N}
\left[ -\log \hat{P}_{\hat{\sigma}(i)}  (c_{i}) +\mathbb{1}_{c_{i}\ne\Phi} L_{\text{box}}(b_{i},\hat{b}_{i}) \right]
$$

数式の(7)部分では、正解クラスラベルの対数確率を求めています。右項はLmatch と同じくバウンディングボックス位置のロスを計算します。つまり LHungarian では、最適なマッチングパターン σ^ からクラスラベルロスとバウンディングボックス位置のロスの和を求め、全オブジェクトの総和を求めています。



# 5. DETR の 微調整 Fine-Tuning

ここからは、本題の DETR の ``Fine-Tuning`` 方法を紹介します。
DETR には ``main.py`` が用意されており、python コマンドで引数を渡すことで学習や推論、``Fine-Tuning`` が可能になっています。
しかし、[COCO dataset](https://arxiv.org/pdf/1512.03385.pdf) 以外を使うことが想定されておらず、``--dataset_file`` 引数で指定できるパラメータが ‘coco’ または 'coco_panoptic’ しかありません。
自分でパラメータを追加してもいいのですが、既に有志の方が ``COCO dataset`` 以外を使えるように修正したソースコードがありましたので、今回はそちらを使った ``Fine-Tuning`` 方法をご紹介したいと思います。
修正版のソースコードは [woctezuma の GitHub](https://github.com/woctezuma/detr/tree/finetune) になります。
また、オリジナルの DETR からの修正箇所は [こちら](https://github.com/woctezuma/detr/compare/master...finetune?diff=split) です。
``--dataset_file`` 引数に 'custom’ を追加されていることが分かるかと思います。


[第1回目の記事](https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/part1.html) と同じく、``Google Colaboratory`` (以下、``Colab``) 上にソースコードを上から順番にコピーすれば動作するように記載していきます。
しかし，推論と違い学習の場合は GPU モードにする必要があります。
以下の画像のように ``Colab`` のメニューの 「ランタイム」→「ランタイムのタイプを変更」 から 「GPU」 に変更してください。
ただし、``Colab`` の ``GPU`` は使いすぎるとアクセス制限がかかることがあります。
今回ご紹介するソースコードを Tesla K80 が割り当てられた Colab で筆者が試したところ、7 epochs 程度でアクセス制限に引っかかってしまいました。
もっと ``Fine-Tuning`` を回したい場合は、AWS や GCP を使う等別途環境を用意する必要があります。


まずは ``Open Image Dataset`` をダウンロードする方法です。
2021 年時点の ``Open Image Dataset`` の最新バージョンは v6 ですが、今回は v4 をダウンロードできるツールである [OIDv4 ToolKit](https://github.com/EscVM/OIDv4_ToolKit) を利用します。
本来は 500GB 超えのデータセットをダウンロードする必要があるので ``Colab`` 上だと展開できないですが、ツールを使うことで必要なクラスラベルを必要な枚数だけダウンロードすることができるので、``Colab`` 上でも問題なくダウンロードすることができます。

では、早速 ``OIDv4 ToolKit`` の Git リポジトリをクローンして必要なパッケージをインストールします。
``requirements.txt`` だけではバージョンの互換性エラーが出るため、``urllib3`` と ``folium`` はバージョンを指定して個別にインストールしています。

In [ ]:
!git clone https://github.com/EscVM/OIDv4_ToolKit.git
!pip install urllib3==1.25.11 folium==0.2.1
!pip install -r OIDv4_ToolKit/requirements.txt

次は、``OIDv4 ToolKit`` を使って画像をダウンロードします。
例として、今回は丸いものを物体検出できるように ``Fine-Tuning`` するために、「Apple」「Ball」「Balloon」「Clock」「Orange」の 5 つのクラスラベルを使うことにします。
いくつかは ``COCO dataset`` にも存在するクラスラベルですが、「Balloon」などは ``OpenImage Dataset`` にしか存在しないクラスラベルになります。
ですので、``COCO dataset`` に存在しない「Balloon」を検出できる、かつ ``COCO dataset`` に存在して今回の 5 つのクラスラベルに存在しないクラス( Umbrella など)が検出できなくなっていれば、``Fine-Tuning`` は成功していると言えます。

また、ダウンロードに時間がかかるため、今回は各クラスのダウンロード枚数を 100 枚に設定して行います。
(その分精度は落ちてしまうので、ちゃんと精度を出したい場合は全データをダウンロードした方が良いです。)


In [ ]:
# ファイル数上限を100枚にするので--limitを指定
!python OIDv4_ToolKit/main.py downloader -y --classes Apple Orange Ball Balloon Clock --type_csv train --limit 100
!python OIDv4_ToolKit/main.py downloader -y --classes Apple Orange Ball Balloon Clock --type_csv validation --limit 100

上記プログラムを実行することで、「/content/OID/Dataset」に画像ファイルがダウンロードされます。
また、OIDv4 ToolKit のオプション引数の詳細は公式の [GitHub](https://github.com/EscVM/OIDv4_ToolKit) に記載されていますが、今回使ったものについて簡単に解説します。

* -y : 不足しているcsvファイルが必要な時、自動的に「yes」を選択する。
* --classes : ダウンロードする対象のクラスラベル(複数指定可能)。
指定できるクラスラベルは、公式サイト の「Category」欄のもの。
* --type_csv : train、validation、test のいずれかを指定。
* --limit :  各クラスラベルのダウンロード上限枚数を指定。

Open Image Dataset をダウンロードできたので、次はアノテーションデータを Open Image format から COCO format に変換を行います。
各フォーマットの詳細は省略しますが、気になる方は kenichiro-yamoto 氏の [はじめての Google Open Images Dataset V6](https://qiita.com/kenichiro-yamato/items/e0c0d6f6138b1c64acd0#oidv6-train-annotations-vrdcsv-%E3%82%92%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89%E3%81%A7%E3%81%8D%E3%82%8B) や、harmegiddo 氏の [COCO Format の作り方](https://qiita.com/harmegiddo/items/da131ae5bcddbbbde41f) で詳しく解説されていますのでご確認ください。
また、GitHub に [openimages2coco](https://github.com/bethgelab/openimages2coco) という変換ツールが公開されていますが、OIDv4 ToolKit を使った場合はうまく動作しませんでしたので、今回は自分で変換プログラムを作成しました。
変換プログラムを作成する際には、soumenpramanik 氏の [Convert-Pascal-VOC-to-COCO](https://github.com/soumenpramanik/Convert-Pascal-VOC-to-COCO/blob/master/convertVOC2COCO.py) を参考にさせていただきました。

In [ ]:
import os, json, glob

def OID2JSON(OIDFiles, saveName, subset):
  """
  アノテーションをOpenImage format(txt)からCOCO format(json)に変換

  Parameters
  ----------
  OIDFiles : string
    OpenImageDatasetのフォルダパス
  saveName : string
    保存ファイル名(json)
  subset : string
    変換したいtype_csv。train、validation、testのいずれかを指定。
  """
  attrDict = dict()
  # categories要素の設定
  attrDict['categories'] = []
  categories = sorted(os.listdir(os.path.join(OIDFiles, 'Dataset', subset)))
  for i in range(len(categories)):
    attrDict['categories'].append({'supercategory': 'none', 'id': i, 'name': categories[i]})

  images = list()
  annotations = list()
  filenames = list()
  image_id = 1
  anno_id = 1
  for category in attrDict['categories']:
    for jpg_file in glob.glob(os.path.join(OIDFiles, 'Dataset', subset, category['name'], '*.jpg')):
      filename = os.path.splitext(os.path.basename(jpg_file))[0]
      # カテゴリ全体で同じファイル名が存在する場合、imageとannoをリネーム
      if filename in filenames:
        rename_filename = filename + '_' + str(image_id)
        os.rename(jpg_file, os.path.join(OIDFiles, 'Dataset', subset, category['name'], rename_filename + '.jpg'))
        os.rename(os.path.join(OIDFiles, 'Dataset', subset, category['name'], 'Label', filename + '.txt'),
                  os.path.join(OIDFiles, 'Dataset', subset, category['name'], 'Label', rename_filename + '.txt'))
        filename = rename_filename
      filenames.append(filename)
      # images要素の設定
      # ※DETRではheightとwidthを使わないので、'none'を設定
      image = {'file_name': filename + '.jpg', 'height': 'none', 'width': 'none', 'id': image_id}
      images.append(image)
      # annotations要素の設定
      anno_path = os.path.join(OIDFiles, 'Dataset', subset, category['name'], 'Label', filename + '.txt')
      with open(anno_path) as f:
        for line in f:
          splitline = line.split(' ')
          # カテゴリがcategories要素に存在しないバウンディングボックスは使わない
          if splitline[0] in [d.get('name') for d in attrDict['categories']]:
            # OpenImageの座標は(xmin, ymin, xmax, ymax)、COCOの座標は(x, y, width, height)
            x1 = int(float(splitline[1]))
            y1 = int(float(splitline[2]))
            x2 = int(float(splitline[3])) - x1
            y2 = int(float(splitline[4])) - y1
            # areaはピクセル数(float)
            area = float(x2 * y2)
            # segmentationは(x1, y1, x2, y2, ...)と順番に定義
            segmentation = [[x1, y1, x1, (y1+y2), (x1+x2), (y1+y2), (x1+x2), y1]]
            annotation = {'iscrowd': 0, 'image_id': image_id, 'bbox': [x1, y1, x2, y2], 'area': area,
                          'category_id': category['id'], 'ignore': 0, 'id': anno_id, 'segmentation': segmentation}
            anno_id += 1
            annotations.append(annotation)
      image_id = image_id + 1

  attrDict['images'] = images
  attrDict['annotations'] = annotations
  attrDict['type'] = 'instances'
  jsonString = json.dumps(attrDict)
  with open(saveName, 'w') as f:
    f.write(jsonString)

上記関数を実行することで Open Image Dataset のアノテーションを、Open Image format(.txt) からCOCO format(.json) に変換することができます。
Open Image format では画像ごとにアノテーションデータが txt ファイルに用意されていますが、COCO format では train, validation, test ごとに 1 つの json ファイルに記載する必要があります。
また、クラスラベルをまたぐと同名のファイルが存在するのでリネームしたり、座標の記載方法が違うので変換する等の処理を行っています。
詳細はプログラムにコメントを記載していますので、そちらをご覧ください。

それでは上記関数を使って、実際に COCO format の json ファイルを生成します。

In [ ]:
OID2JSON('/content/OID', 'custom_train.json', 'train')
OID2JSON('/content/OID', 'custom_val.json', 'validation')

これで COCO format のアノテーションデータの準備は完了です。

## 5-2.データセットのディレクトリ構成変更

前章はフォーマット変換を行いました。
次は、画像ファイルや前章で作成したアノテーションデータを指定のディレクトリに配置します。
元々が COCO dataset を使うように想定されているため、ディレクトリ構成も COCO dataset に合わせる必要があるためです。
ディレクトリ構成は DETR の GitHub にも記載がありますが、以下のような構成になります。

```
# path/to/coco/
# ├ annotations/  # JSON annotations
# │  ├ custom_train.json
# │  └ custom_val.json
# ├ train2017/    # training images
# └ val2017/      # validation images
```

それでは、「/content/data」フォルダ内にファイルを移動してみましょう。

In [ ]:
import shutil

source_train_paths = glob.glob(os.path.join('/content/OID/Dataset', 'train', '**/'))
source_val_paths = glob.glob(os.path.join('/content/OID/Dataset', 'validation', '**/'))
train_path = '/content/data/custom/train2017/'
val_path = '/content/data/custom/val2017/'
convert_anno_path = '/content/data/custom/annotations/'
# ディレクトリ作成
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(convert_anno_path, exist_ok=True)
# train移動
for source_train_path in source_train_paths:
  for img_path in glob.glob(os.path.join(source_train_path, '*.jpg')):
    shutil.move(img_path, train_path)
# val移動
for source_val_path in source_val_paths:
  for img_path in glob.glob(os.path.join(source_val_path, '*.jpg')):
    shutil.move(img_path, val_path)
# anno移動
shutil.move('/content/custom_train.json', convert_anno_path)
shutil.move('/content/custom_val.json', convert_anno_path)

上記プログラムを実行すれば、ディレクトリ構成変更作業は完了です。
`/content/data/train2017/` 以下にクラスラベル関係なく画像ファイルが置かれていて、 `/content/data/annotations/` 以下に前章で作成したアノテーションファイルが置かれていれば OK です。

In [ ]:
!ls /content/data/custom
!ls /content/detr

# 5-3.Fine-Tuning

前章でデータセットの準備は完了したので、本章では Fine-Tuning について解説します。
まずは必要なパッケージやライブラリを import します。
ここで注意なのが、PyTorch のバージョンが 1.9.0 以上だと Fine-Tuning 時にエラーとなってしまいました。
ですので、1.8.0 にダウングレードを行っています。



In [ ]:
!pip install -q torch==1.8.0 torchvision==0.9.0 torchtext==0.9.0

import torch, torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from PIL import Image
import requests
print(torch.__version__)           # 1.8.0
print(torchvision.__version__)     # 0.9.0
print(torch.cuda.is_available())   # True
torch.set_grad_enabled(False);

次に、COCO dataset 以外に対応した、woctezuma 氏の DETR をチェックアウトします。
ブランチも切り替える必要があるので、finetune ブランチに切り替えています。

In [ ]:
%cd /content/
!rm -rf detr
!git clone https://github.com/woctezuma/detr.git
# ブランチの切替
%cd detr/
!git checkout finetune

/content に detr というフォルダができていれば成功です。
次は、学習済みモデルをダウンロードして、それをベースに Fine-Tuning を行います。
学習済みモデルは、Model Zoo に公開されている「DETR-DC5」を使います。
また、このモデルは COCO dataset で使う想定であるため、COCO dataset のクラス数である 92 クラスに分類するようになっています。
今回は 6 クラス (Open Image Dataset の 5 クラス＋no-object) に分類したいため、該当箇所の重みを削除して一から学習する必要があります。
その方法は DETR の issues に載っていましたので、そちらを参考にしながらコーディングを進めます。
下記ソースコードを実行すると、学習済みモデルのパラメータから不要なパラメータを削除したモデルを 「detr-r50_no-class-head.pth」 というファイル名で保存します。

In [ ]:
# 学習済みモデルの取得
checkpoint = torch.hub.load_state_dict_from_url(
    url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
    map_location='cpu',
    check_hash=True
)

# 分類ヘッドの削除
del checkpoint['model']['class_embed.weight']
del checkpoint['model']['class_embed.bias']

# 保存
torch.save(checkpoint, 'detr-r50_no-class-head.pth')

次はいよいよ Fine-Tuning です。
クラスラベルの数を指定する必要があるので、``main.py`` を実行する前に定義しています。
ここで注意ですが no-object は除いたクラスラベル数を定義する必要があります。
no-object 自体は、DETR のプログラム内で自動的に「num_classes」と同じ ID で採番されるためです。
(つまり num_classes=5 の場合、'Apple’=0, 'Ball’=1, 'Balloon’=2, 'Clock’=3, 'Orange’=4, 'no-object’=5 のように ID が採番されます)

Fine-Tuning を行うと処理に時間がかかりますので、とりあえず下記に示すソースコードでは、10 epochs だけ学習するようにしています。
精度が高いモデルを作りたい場合は、 OIDv4 ToolKit の ``--limit`` オプションを撤廃して画像枚数を増やしたり、``--epochs`` オプションの値を大きくして学習回数を増やしてみてください。
その分処理時間がかかってしまいますので、そこはご注意ください。

補足として、筆者が試した場合の処理時間を記載します。また、Colab の GPU は Tesla K80 が割り当てられている場合になります。

* ``--limit=100`` : 1 epoch に 5 分程度。
* ``--limit`` なし : 1 epoch に 1 時間弱。

In [ ]:
num_classes = 5

%cd /content/detr/
os.makedirs('outputs', exist_ok=True)

# 学習
!python main.py \
    --dataset_file "custom" \
    --coco_path "/content/data/custom/" \
    --output_dir "outputs" \
    --resume "detr-r50_no-class-head.pth" \
    --num_classes $num_classes \
    --epochs 10

エラーが発生せず学習が進んでいれば、 Fine-Tuning は成功です。
しかし、Colab で実行すると [GPU の使用制限](https://note.com/npaka/n/n1aa6f8c973d0) に引っかかってしまう等の問題があり、筆者が ``--limitなし`` で Colab で試したところ、7 epochs 程度しか学習できませんでした。
実際に Fine-Tuning を何十 epochs も回す場合は、``checkpoint`` と ``resume`` を実装して途中保存しながら学習するか、AWS や GCP 等の環境を使った方が良さそうです。

# 5-4. Fine-Tuning 結果の確認

本章では、Fine-Tuning を行ったモデルを使って正しく学習できているか、正しく物体検出できているかを確認します。
モデルについては、前章で説明したとおり Colab では厳しかったため、GCP に環境を用意して ``--limitなし`` かつ 50 epochs 程度学習したモデルを使って結果を確認します。
モデルは ``/content/detr/outputs`` に配置されているものとし、解説していきます。

まずはモデルの ``loss`` と ``mAP`` を確認してみます。
``mAP`` は物体検知モデルに使われる評価指標であり、``mAP`` が高いほどモデルの精度が高く常に自信があるモデルといえます。
本記事では ``mAP`` について解説はしませんが、Jonathan Hui 氏の [mAP(mean Average Precision) for Object Detection](https://jonathan-hui.medium.com/map-mean-average-precision-for-object-detection-45c121a31173) という記事が非常に分かりやすかったので、興味のある方はこちらをご確認いただければと思います。
``loss`` と ``mAP`` の確認方法ですが、DETR では既に確認用プログラムを作ってくださっているので、そちらを使って確認していきます。

In [ ]:
from util.plot_utils import plot_logs
from pathlib import Path

%cd /content/detr/
log_directory = [Path('/content/detr/outputs')]

# 実線 ... トレーニング結果(train_loss)
# 破線 ... 検証結果(val_loss)
fields_of_interest = (
    'loss',
    'mAP',
)
plot_logs(log_directory, fields_of_interest)

上記プログラムを実行すると、下画像のようなグラフが出力されます。
``loss`` を見てみると、``train_loss`` は学習が進むにつれて下がっているので正しく学習できていそうですし、``val_loss`` も上がっていないので過学習にもなっていないことが分かります。
``mAP`` を見ても、学習が進むにつれて右肩上がりになっています。
正しく学習できていますが、まだ収束していないので、もう少し学習を回した方が良かったかもしれません。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-023.png" width="49%">
</center>

次は、実際に画像を入力して物体検出をしてみます。
最初にモデルをロードします。
``Fine-Tuning`` したモデルと ``Fine-Tuning`` していないモデルを比較したいので、2 種類のモデルをロードしておきます。
``Fine-Tuning`` していないモデルに関しては、[第 1 回目の記事](https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/part1.html) で使用したモデルを使います。

In [ ]:
finetuned_model = torch.hub.load('facebookresearch/detr',
                       'detr_resnet50',
                       pretrained=False,
                       num_classes=num_classes)
checkpoint = torch.load('/content/detr/outputs/checkpoint.pth',
                        map_location='cpu')
finetuned_model.load_state_dict(checkpoint['model'], strict=False)
finetuned_model.eval()

original_model = torch.hub.load('facebookresearch/detr', 'detr_resnet50_dc5', pretrained=True)
original_model.eval()

モデルの準備が完了したので、次は実際に画像を入力して物体検出を行うための前処理や画像表示処理を定義します。
なお前処理等は、[第1回目の記事](https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/part1.html) で紹介した内容と同じなので、詳細な解説は前回の記事を見てください。




In [ ]:
# 可視化用クラスラベル
oid_labels = [
  'Apple',
  'Ball',
  'Balloon',
  'Clock',
  'Orange',
]
coco_labels = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]
# 可視化用COLOR
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# 標準的なPyTorchのmean-std入力画像の正規化
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def box_cxcywh_to_xyxy(x):
    """
    (center_x, center_y, width, height)から(xmin, ymin, xmax, ymax)に座標変換
    """
    # unbind(1)でTensor次元を削除
    # (center_x, center_y, width, height)*N → (center_x*N, center_y*N, width*N, height*N)
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    # (center_x, center_y, width, height)*N の形に戻す
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    """
    バウンディングボックスのリスケール
    """
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    # バウンディングボックスの[0～1]から元画像の大きさにリスケール
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def filter_bboxes_from_outputs(outputs, threshold=0.7):
  # 閾値以上の信頼度を持つ予測値のみを保持
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold
  probas_to_keep = probas[keep]
  # [0, 1]のボックスを画像のスケールに変換
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  return probas_to_keep, bboxes_scaled

# 結果の表示
def plot_finetuned_results(pil_img, prob=None, boxes=None, labels=None):
  plt.figure(figsize=(16, 10))
  plt.imshow(pil_img)
  ax = plt.gca()
  colors = COLORS * 100
  if prob is not None and boxes is not None:
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
      ax.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin,
                                 fill=False, color=c, linewidth=3))
      cl = p.argmax()
      print(labels, p)
      text = f'{labels[cl]}: {p[cl]:0.2f}'
      ax.text(xmin, ymin, text, fontsize=15,
              bbox=dict(facecolor='yellow', alpha=0.5))
  plt.axis('off')
  plt.show()

# 物体検出
def run_worflow(my_image, my_model, labels, threshold=0.7):
  # mean-std入力画像の正規化(バッチサイズ : 1)
  img = transform(my_image).unsqueeze(0)
  # モデルに反映
  outputs = my_model(img)

  probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs, threshold=threshold)
  plot_finetuned_results(my_image, probas_to_keep, bboxes_scaled, labels)

これで物体検出の準備ができました。
それでは、実際に物体検出をしてみましょう。
試しに、``Open Image Dataset`` から ``Balloon`` の画像をダウンロードして物体検出してみます。
``Fine-Tuning`` したモデルでは検出できて、``Fine-Tuning`` していないモデルでは検出できないはずです。
また、閾値は 0.9 を指定しています。

In [ ]:
from io import BytesIO

url = 'https://farm7.staticflickr.com/52/106887535_a29c34113b_o.jpg'
response = requests.get(url)
im = Image.open(BytesIO(response.content))

# Fine-Tuningモデルで物体検出(閾値0.9)
run_worflow(im, finetuned_model, oid_labels, 0.9)
# Fine-Tuningしていないモデルで物体検出(閾値0.9)
run_worflow(im, original_model, coco_labels, 0.9)

実行すると、下画像のような結果が表示されます。
左側が ``Fine-Tuning`` したモデル、右側が ``Fine-Tuning`` していないモデルです。
想定通り、``Fine-Tuning`` した場合は ``Balloon`` が検出できていますが、``Fine-Tuning`` していない場合は検出できていません。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-024.png" width="33%">
</center>

``Open Image Dataset`` や ``COCO dataset`` の画像を使い色々なパターンを試してみたので、以下に結果を列挙します。
上記画像と同じく、左側が ``Fine-Tuning`` したモデル、右側が ``Fine-Tuning`` していないモデルとなります。
``Fine-Tuning`` で学習しているりんごやオレンジ、アナログ時計に関しては ``Fine-Tuning`` 前と同程度の検出精度ですが、学習していない傘や人や犬、デジタル時計などは検出しないようになっていて、正しく ``Fine-Tuning`` できていることが分かるかと思います。

<center>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-025.png" width="49%">
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-026.png" width="49%">
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-027.png" width='49%'>
<img src="https://www.ogis-ri.co.jp/otc/hiroba/technical/detr/img/pic202111-028.png" width="49%">
</center>



# 6.おわりに

今回の記事では、前半では DETR の詳細と自然言語処理との Transformer の違いを解説し、後半は DETR での ``Fine-Tuning`` 方法と結果を確認しました。
DETR の Fine-Tuning 方法を紹介している記事が無さそうでしたので、誰かのお役に立てれば幸いです。
また、DETR だからこそできる物体検出があるわけではなく、既存の FasterR-CNN と同程度の性能ですので、案件への応用例などは既存手法と変わりません。
しかし、既存手法と全く違うアーキテクチャでありながら、既存手法と同程度の性能が出せることが非常に素晴らしいので、今後 DETR をベースにした手法が出てくることでしょう。

今回で DETR の記事は終了です。
次回の記事では、DETR と自然言語処理で使われる RoBERTa を組み合わせた、[MDETR](https://arxiv.org/pdf/2104.12763.pdf) を紹介予定です。
画像とテキストを混ぜたマルチモーダル推論モデルで、両方とも ``Transformer`` を使うため既存手法よりも自由度が高い推論ができるものになっています。

